In [12]:
import gym
import numpy as np
import sys
from collections import defaultdict
from IPython.display import clear_output
import time
import math

# Utils

In [13]:
def serialize_Q(Q, name):
    file = open(name, 'w+')
    for key in Q.keys():
        line = [f"{key}"]
        for value in Q[key]:
            line.append(f"{value}")
        file.write(';'.join(line) + '\n')
    file.close()
    
def deserialize_Q(name):
    file = open(name, 'r')
    Q = {}
    for line in file.readlines():
        line_arr = line[:-1].split(';')
        key = int(line_arr.pop(0))
        Q[key] = np.array([float(e) for e in line_arr])
    file.close()
    return Q

In [14]:
def generate_empty_dictionary(env, default_value=0.0):
    return {k:np.ones(env.action_space.n)*default_value for k in range(env.observation_space.n)}

In [42]:
def greedy_policy_from_Q(Q):
    return {k:np.argmax(v) for k, v in Q.items()}

def epsilon_greedy_for_state_action_values(Q_s, epsilon, env):
    policy_s = np.ones(env.action_space.n) * epsilon / env.action_space.n
    best_a = np.argmax(Q_s)
    policy_s[best_a] = 1 - epsilon + (epsilon / env.action_space.n)
    return policy_s

def generate_episode(env, Q=None, epsilon=None):
    greedy_policy = greedy_policy_from_Q(Q) if Q else None
    episode = []
    state = env.reset()
    while True:
        if Q is None:
            action = env.action_space.sample()
        elif Q is not None and epsilon is not None:
            action = np.random.choice(np.arange(env.action_space.n), p=epsilon_greedy_for_state_action_values(Q[state], epsilon, env)) if state in Q else env.action_space.sample()
        else:
            action = greedy_policy[state]
        next_state, reward, done, info = env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if done:
            break
    return episode

In [43]:
def render_playthrough(env, Q=None, epsilon=None):
    greedy_policy = greedy_policy_from_Q(Q) if Q else None
    state = env.reset()
    rewards = 0
    clear_output(True)
    env.render()
    time.sleep(.3)
    while True:
        if greedy_policy is None:
            action = env.action_space.sample()
        elif Q is not None and epsilon is not None:
            action = np.random.choice(np.arange(env.action_space.n), p=epsilon_greedy_for_state_action_values(Q[state], epsilon, env)) if state in Q else env.action_space.sample()
        else:
            action = greedy_policy[state]
        next_state, reward, done, info = env.step(action)
        rewards += reward
        state = next_state
        clear_output(True)
        env.render()
        time.sleep(.3)
        if done:
            print(rewards)
            break

In [44]:
def benchmark(env, Q=None, epsilon=None):
    greedy_policy = greedy_policy_from_Q(Q) if Q else None
    all_rewards = []
    for i in range(100):
        state = env.reset()
        rewards = 0
        while True:
            if Q is None:
                action = env.action_space.sample()
            elif Q is not None and epsilon is not None:
                action = np.random.choice(np.arange(env.action_space.n), p=epsilon_greedy_for_state_action_values(Q[state], epsilon, env)) if state in Q else env.action_space.sample()
            else:
                action = greedy_policy[state]
            next_state, reward, done, info = env.step(action)
            rewards += reward
            state = next_state
            if done:
                all_rewards.append(rewards)
                break
    return np.average(np.array(all_rewards))

# MC Prediction

In [45]:
def mc_prediction_q(env, num_episodes, gamma=1.0):
    # initialize empty dictionaries of arrays
    returns_sum = generate_empty_dictionary(env)
    N = generate_empty_dictionary(env)
    Q = generate_empty_dictionary(env)
    # loop over episodes
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 100 == 0:
            clear_output(True)
            print("\rEpisode {}/{}.\n".format(i_episode, num_episodes), end="")
        
        ## TODO: complete the function
        episode = generate_episode(env)
        states = list(map(lambda x: x[0], episode))
        rewards = np.array(list(map(lambda x: x[2], episode)))
        discounts = np.array([gamma ** (i + 1) for i in range(0, len(rewards))])
        for i in range(0, len(episode)):
            s_i, a_i, r_i_next = episode[i]
            if s_i not in states[0:i]:
                N[s_i][a_i] += 1
                returns_sum[s_i][a_i] += r_i_next + gamma * sum(rewards[i + 1:] * discounts[i + 1:])
    
    for state in returns_sum.keys():
        for i in range(len(returns_sum[state])):
            if N[state][i] != 0.0:
                Q[state][i] = returns_sum[state][i] / N[state][i]
    
    return Q

In [46]:
env = gym.make('Taxi-v2')
render_playthrough(env)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
-623


In [47]:
Q = mc_prediction_q(env, 10000)
for i in range(10):
    print(f"Action values for {i} state: {Q[i]}")

Episode 10000/10000.
Action values for 0 state: [0. 0. 0. 0. 0. 0.]
Action values for 1 state: [-492.74107143 -420.60869565 -427.84       -464.63963964 -406.37614679
 -425.39583333]
Action values for 2 state: [-474.52777778 -431.76033058 -432.42056075 -413.375      -422.31192661
 -455.10091743]
Action values for 3 state: [-469.86238532 -434.71698113 -459.67592593 -432.75384615 -450.35964912
 -468.64285714]
Action values for 4 state: [-549.52380952 -505.60576923 -506.18556701 -495.21212121 -526.75
 -491.18292683]
Action values for 5 state: [0. 0. 0. 0. 0. 0.]
Action values for 6 state: [-548.12698413 -463.67045455 -527.26804124 -504.66346154 -496.07058824
 -517.52808989]
Action values for 7 state: [-554.08045977 -510.63809524 -532.05405405 -537.57692308 -531.36
 -504.58888889]
Action values for 8 state: [-480.55789474 -482.54761905 -486.97674419 -465.71287129 -489.39473684
 -476.67619048]
Action values for 9 state: [-477.74736842 -498.1686747  -480.94230769 -501.31578947 -465.88372093
 

In [48]:
Q_1 = mc_prediction_q(env, 1000)
Q_2 = mc_prediction_q(env, 2000)
Q_3 = mc_prediction_q(env, 3000)

print(benchmark(env, Q_1))
print(benchmark(env, Q_2))
print(benchmark(env, Q_3))

Episode 3000/3000.
-1133.21
-1150.94
-953.57


## MC Control

In [49]:
def mc_control(env, num_episodes, alpha, eps=0.1, gamma=1.0):
    nA = env.action_space.n
    # initialize empty dictionary of arrays
    Q = generate_empty_dictionary(env)
    # loop over episodes
    eps_a = 1.0
    
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 100 == 0:
            clear_output(True)
            print("\rEpisode {}/{}.\n".format(i_episode, num_episodes), end="")
        
        ## TODO: complete the function
        eps_a = max(eps_a * 0.9999, eps)
        episode = generate_episode(env, Q, eps_a)
        
        states = list(map(lambda x: x[0], episode))
        rewards = np.array(list(map(lambda x: x[2], episode)))
        discounts = np.array([gamma**i for i in range(len(rewards)+1)])
        
        for i in range(0, len(episode)):
            s_i, a_i, r_i_next = episode[i]
            if s_i not in states[0:i]:
                Q[s_i][a_i] = Q[s_i][a_i] + alpha * (sum(rewards[i:]*discounts[:-(1+i)]) - Q[s_i][a_i])
            
    return Q

In [50]:
Q = mc_control(env, 10000, 0.01)

Episode 10000/10000.


In [51]:
print(benchmark(env, Q))

-400.03


In [53]:
render_playthrough(env, Q)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
-200


## TD Learning (Sarsa)

In [60]:
def sarsa(env, num_episodes, alpha, gamma=1.0):
    Q = generate_empty_dictionary(env)
    
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 100 == 0:
            clear_output(True)
            print("\rEpisode {}/{}.\n".format(i_episode, num_episodes), end="")
        
        state = env.reset()  
        epsilon = 1.0 / i_episode
        action = np.random.choice(np.arange(env.action_space.n), p=epsilon_greedy_for_state_action_values(Q[state], epsilon, env))

        for t_step in np.arange(300):
            next_state, reward, done, info = env.step(action)
            if not done:
                next_action = np.random.choice(np.arange(env.action_space.n), p=epsilon_greedy_for_state_action_values(Q[next_state], epsilon, env))
                Q[state][action] = Q[state][action] + alpha * (reward + (gamma * Q[next_state][next_action]) - Q[state][action])
                state = next_state
                action = next_action
            if done:
                Q[state][action] = Q[state][action] + alpha * (reward + (gamma * 0) - Q[state][action])
                break

    return Q

In [76]:
Q = sarsa(env, 200000, 0.03)

Episode 200000/200000.


In [78]:
print(benchmark(env, Q))

8.24


In [79]:
render_playthrough(env, Q)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
8


## TD Learning (Q-Learning)

In [80]:
def q_learning(env, num_episodes, alpha, gamma=1.0):
    Q = generate_empty_dictionary(env)
    
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 100 == 0:
            clear_output(True)
            print("\rEpisode {}/{}.\n".format(i_episode, num_episodes), end="")
        
        state = env.reset()  
        epsilon = 1.0 / i_episode
        action = np.random.choice(np.arange(env.action_space.n), p=epsilon_greedy_for_state_action_values(Q[state], epsilon, env))

        for t_step in np.arange(300):
            next_state, reward, done, info = env.step(action)
            if not done:
                next_action = np.random.choice(np.arange(env.action_space.n), p=epsilon_greedy_for_state_action_values(Q[next_state], epsilon, env))
                Q[state][action] = Q[state][action] + alpha * (reward + (gamma * np.amax(Q[next_state])) - Q[state][action])
                state = next_state
                action = next_action
            if done:
                Q[state][action] = Q[state][action] + alpha * (reward + (gamma * 0) - Q[state][action])
                break

    return Q

In [83]:
Q = q_learning(env, 200000, 0.01)

Episode 200000/200000.


In [84]:
print(benchmark(env, Q))

8.63


In [85]:
render_playthrough(env, Q)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
7
